# E2E 테스트 – Product RAG PoC  

1. '.env' 로부터 OpenAI 키 로딩  
2.  woori_deoposit → FAISS 인덱스 생성  
3. LangGraph product 파이프라인 로드  
4. 샘플 질의에 대한 응답 생성  
5. RAGAS 평가로 정확도·신뢰도 확인

In [2]:
# 1. env & 라이브러리
from pathlib import Path
from dotenv import load_dotenv
import os, json, pprint, sys, pandas as pd

ROOT = Path.cwd().parent  # 노트북은 tests/ 하위에 있다고 가정
sys.path.append(str(ROOT))

load_dotenv(ROOT / ".env")
print(f"ROOT - {ROOT}")
print("환경 변수 로드 완료")


ROOT - c:\Users\insung\Finance_Agent
환경 변수 로드 완료


In [2]:
# 2. 인덱스 준비
from scripts.build_deposit_index import build_index

build_index()

meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '수수료'} - chunk lenght: 3
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '중도해지'} - chunk lenght: 1
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '중도해지'} - chunk lenght: 6
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '가입방법'} - chunk lenght: 3
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '수수료'} - chunk lenght: 1
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '수수료'} - chunk lenght: 1
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', 'section': '수수료'} - chunk lenght: 3
meta_base: {'product_code': 'PLM_P00XXXXXXX', 'file_date': '2025-03-31', 'product': '- 1 -', '

  0%|          | 0/6 [00:00<?, ?it/s]

2025-07-30 13:10:31,908 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 13:10:33,825 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 13:10:36,257 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 13:10:38,498 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 13:10:40,431 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 13:10:41,926 | INFO | HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-07-30 13:10:42,123 | INFO | Loading faiss with AVX2 support.
2025-07-30 13:10:42,173 | INFO | Successfully loaded faiss with AVX2 support.
2025-07-30 13:10:42,192 | INFO | Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
2025-07-30 

> 1,095 chunks indexed


In [3]:
# 3. 파이프라인 로딩
from services.rag.product_rag.chain import graph

sample_q = "우리사랑정기적금 3년 만기 기본금리가 얼마나 돼?"
result = graph.invoke({"question": sample_q})
pprint.pp(result)


{'question': '우리사랑정기적금 3년 만기 기본금리가 얼마나 돼?',
 'answer': '우리사랑정기적금의 3년 만기 기본금리는 연 2.2%입니다(①4).',
 'context': '기본이율×80%×보유일수/계약일수\n'
            '11개월 이상 ~ 3년 미만\n'
            '기본이율×90%×보유일수/계약일수\n'
            '3년 이상\n'
            '기본이율\n'
            '  ※ 기본이율 : 신규(또는 재예치)시점 해당 상품의 기본금리\n'
            '구 분\n'
            '내 용\n'
            '\n'
            '구분\n'
            '1개월\n'
            '3개월\n'
            '6개월\n'
            '12개월\n'
            '24개월\n'
            '36개월\n'
            '기본금리\n'
            '연 2.46%\n'
            '연 2.49%\n'
            '연 2.45%\n'
            '연 2.49%\n'
            '연 2.37%\n'
            '연 2.49%\n'
            '※ 현재 적용금리 확인 : 우리은행 인터넷 홈페이지(www.wooribank.com)\n'
            '구 분\n'
            '내 용\n'
            '이자지급시기\n'
            '• 만기일시지급식(단리식) : 만기(후) 또는 중도해지 요청시 이자를 지급 \n'
            '만기이자 \n'
            '산출근거\n'
            '• 신규금액 × 약정이율 × (계약일수 ÷ 365일)\n'
            ' ※ 계약일수는 신규일부터 해지일 전일까지 일수를 의미\n'
            ' [예시] 신규일

## 배치 테스트 & RAGAS 평가  
- 10개 샘플 질의를 선택해 파이프라인을 실행  
- answer relevancy / faithfulness / LLMContextPrecisionWithoutReference으로 지표 계산
- 레퍼런스가 없기 때문에 LLMContextPrecisionWithoutReference으로 답변과 컨텐스트 정보의 중복 정도로 precision 평가

In [4]:
# 4. 배치 & RAGAS
from ragas import EvaluationDataset, SingleTurnSample, evaluate
from ragas.metrics import answer_relevancy, faithfulness, LLMContextPrecisionWithoutReference
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 1) 평가용 LLM/임베딩 (Ragas 래퍼로 감싸기)
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini", temperature=0))
evaluator_emb = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

# 2) 그래프 호출 → Ragas 샘플로 변환
SAMPLE_QUESTIONS = [
    # ── 우리 SUPER주거래 적금 ─────────────────────────────
    "우리 SUPER주거래 적금의 3년 만기 기본금리는 얼마인가요?",
    "SUPER주거래 적금에서 우대금리를 받으려면 어떤 거래실적을 충족해야 해?",

    # ── 주택청약예금 ────────────────────────────────────
    "주택청약예금은 누가 가입할 수 있고 최소 가입금액은 얼마인가요?",
    "주택청약예금을 중도해지하면 이율이 어떻게 적용되나요?",

    # ── 우리 자유적금 ───────────────────────────────────
    "우리 자유적금의 최대 월 납입금액이 궁금해요.",
    "우리 자유적금에서 중도해지 시 적용되는 이율은?",

    # ── 정기적금 ────────────────────────────────────────
    "정기적금 3년제 기본금리는 현재 얼마인가요?",
    "정기적금을 만기 지난 뒤 해지하면 어떤 만기후이율이 적용되나요?",

    # ── 첫급여 우리적금 ─────────────────────────────────
    "첫급여 우리적금은 어떤 사람이 가입할 수 있나요?",
    "첫급여 우리적금에서 우대이율을 받기 위한 조건은 무엇인가요?",
]

samples = []
for q in SAMPLE_QUESTIONS:
    res = graph.invoke({"question": q})
    ans = res.get("answer", "")
    ctxs = res.get("contexts", [])
    if isinstance(ctxs, str):
        ctxs = [c for c in ctxs.split("\n\n") if c.strip()]

    samples.append(
        SingleTurnSample(
            user_input=q,
            response=ans,
            retrieved_contexts=ctxs,
        )
    )

dataset = EvaluationDataset(samples=samples)

# 3) 메트릭 설정 (레퍼런스 없이 가능한 조합)
metrics = [
    answer_relevancy,                                       # Q - A 관련성
    faithfulness,                                           # A가 컨텍스트에 근거?
    LLMContextPrecisionWithoutReference(llm=evaluator_llm)  # 컨텍스트가 Q와 관련?
    ]

# 4) 평가 실행
report = evaluate(
    dataset,
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=evaluator_emb,
    show_progress=True,
    batch_size=8,
)

df_scores  = report.to_pandas()

df_overall = df_scores[[
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]].mean()

print("=== Overall Metrics ===")
print(df_overall)

print("\n=== Sample-level Metrics (sorted by Faithfulness) ===")
print(df_scores.sort_values("faithfulness")[[
    "user_input",
    "retrieved_contexts",
    'response',
    "answer_relevancy",
    "faithfulness",
    "llm_context_precision_without_reference"
]])

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Batch 1/4:   0%|          | 0/8 [00:00<?, ?it/s]

=== Overall Metrics ===
answer_relevancy                           0.393057
faithfulness                               0.833333
llm_context_precision_without_reference    0.736000
dtype: float64

=== Sample-level Metrics (sorted by Faithfulness) ===
                                  user_input  \
0         우리 SUPER주거래 적금의 3년 만기 기본금리는 얼마인가요?   
2        주택청약예금은 누가 가입할 수 있고 최소 가입금액은 얼마인가요?   
1  SUPER주거래 적금에서 우대금리를 받으려면 어떤 거래실적을 충족해야 해?   
3              주택청약예금을 중도해지하면 이율이 어떻게 적용되나요?   
4                  우리 자유적금의 최대 월 납입금액이 궁금해요.   
5                 우리 자유적금에서 중도해지 시 적용되는 이율은?   
6                   정기적금 3년제 기본금리는 현재 얼마인가요?   
7        정기적금을 만기 지난 뒤 해지하면 어떤 만기후이율이 적용되나요?   
8                첫급여 우리적금은 어떤 사람이 가입할 수 있나요?   
9          첫급여 우리적금에서 우대이율을 받기 위한 조건은 무엇인가요?   

                                  retrieved_contexts  \
0  [및 특징\n⋅상 품 명 : 「우리 SUPER주거래 적금」\n⋅상품특징 : 우리은행...   
2  [및 특징\n  ￭ 상품명 : 주택청약예금\n  ￭ 상품특징 : 민영주택 또는 민간...   
1  [및 특징\n⋅상 품 명 : 「우리 SUPER주거래 적금」\n⋅상품특징 : 우리은행... 

In [5]:
for i in range(10):
    print('\nuser_input:',df_scores['user_input'][i])
    print('retrieved_contexts:',df_scores['retrieved_contexts'][i])
    print('response:',df_scores['response'][i])
    print('='*120)


user_input: 우리 SUPER주거래 적금의 3년 만기 기본금리는 얼마인가요?
retrieved_contexts: ['및 특징\n⋅상 품 명 : 「우리 SUPER주거래 적금」\n⋅상품특징 : 우리은행 거래실적에 따라 우대금리를 제공하는 적립식 상품', '기본금리\n• 연 0.1% (세급납부 전 2025.06.10.기준)\n ※ 적용이자율은 매일 변경 고시되며 금리 변경 시에는 변경 고시일부터 변경 적용됩니다.\n우대금리\n• ‘CJ PAY 우리 통장 서비스’ 이용 시, 매일의 최종 잔액 중 2백만원 이하\n  금액에 대해서 아래의 기간 동안 우대금리 및 이벤트 특별금리를 제공합니다.\n• 해지 후 재가입 시 우대금리 및 이벤트 특별금리를 제공하지 않습니다. \n단, 최초 가입일로부터 1개월 이내 재가입한 경우에는 제공합니다.\n구분\n적용요건\n우대기간\n우대금리\n일반 \n우대금리\nCJ PAY 우리 통장 서비스 \n이용시 \n가입일로부터 1년 이내\n연 1.90%p\n가입후 1년 초과 5년 이내\n연 0.40%p\n이벤트 \n특별금리\nCJ PAY 우리 통장 서비스', '달라질 수 있습니다.\n⋅만기 전 해지할 경우 약정한 이율보다 낮은 중도해지이율이 적용됩니다.\n⋅우대금리는 만기 해지 시에만 적용되며, 중도해지하는 경우 적용되지 않습니다.\n⋅’19.7.19.부터 “위비SUPER주거래적금Ⅱ”에서 “우리 SUPER주거래 적금”으로 변경되었습니다.\n  기존에 “위비SUPER주거래적금Ⅱ”를 가입한 고객님께서도 상품명이 “우리 SUPER주거래 적금”  \n  으로 변경되며, 우대이율은 시행일 이후 가입자에 대해서 변경된 상품 내용을 \n  적용받게 됩니다.\n⋅금리변동형 정기적금은 금리상승기에 약정금리 인상 가능성이 큰 반면, 금리 하락기에는 \n   약정금리 인하로 소비자에게 불리할 수 있습니다.', '※ 적용세율은 2024.12.12. 현재의 세율이며, 관계법령의 개정에 따라 변경될 수 있음\n구 분\n내 용\n기본이율\n(세금납부 전\n’24.1

- answer_relevancy ≥ 0.8 이면 FAQ 일치율 양호   낮으면 프롬프트/쿼리 확장/리트리버 k값 조정.

- faithfulness < 0.9 라면 Prompt 수정·retriever k값 조정 필요, ‘컨텍스트 내 근거 인용’ 지시 강화, 답변 길이 제한, 시스템 프롬프트 보강.

- context_precision 은 1.0에 가까울수록 불필요한 문맥이 적음. 임베딩 모델/토크나이저, 인덱스 파라미터(HNSW, k), 쿼리 재작성 확인.

 OOD(미지원 질문)용 거절 정확도 같은 보조 지표도 추가하면 좋을 듯.